# Testing Software

⏱️ 30 min.

Writing code is step one. Step two is making sure that the code you're writing is correct. 

Of course, this is a process that you have gone through when using Python (or Excel) to complete a process. When programming, it's best practice to encode these checks into `Tests`. 

Software testing is a tool we can use to:
1. Build confidence that a process is automated correctly
2. Allow us to make changes to the process without breaking it
3. Help other people understand expected behavior of our automation

If a process is something you're using on a go-forward basis, you probably want some tests for it!

## Unit Tests


Unit tests are just tests for specific chunks (aka units!) of code. A unit test ensures that the specific ode performs as expected and handles various scenarios correctly.

Consider the `add(x, y)` function below.

In [ ]:
def add(x, y):
    return x + y

💡 A **function** is often the most useful boundry to unit test on. If you've broken your functions up effectively, then each function should be doing a single, coherent chunk of work. 

As such, it makes sense to test the behavior of the function as a single unit!

In [ ]:
# Test a single addition works
add(1, 2) == 3

`assert` is a Python keyword that checks that the value passed to it is True, and optionally allows you to give an error message.

In [ ]:
assert True, 'Asserting True should pass. Nothing should be printed after this cell.'

In [ ]:
assert False, 'Asserting False should fail. We should get an error message after we run this cell.'

💡 When we test functions, we want to test:
- The most common behaviors
- Edge cases that we are likely to run into

Overall, our goal isn't to write tests that pass easily. **Our goal is to write tests that probe the function, and ensure that is correct**. Don't just phone it in on tests. If you write good tests, you should be nervous to run them because they actually might fail and catch your bugs!

In [ ]:
# Test common cases
assert add(1, 2) == 3, 'Adding positive numbers failed'
assert add(-1, -2) == -3, 'Adding negative numbers failed'
assert add(-1, 2) == 1, 'Adding a negative and positive number failed'

# Some edge cases
assert add(1, 2.2) == 3.2, 'Adding an integer and float failed'

🧑‍💻 Write a test that tests another case you're likely to encounter. Anything we might be missing with our tests?

In [ ]:
# TODO: Write your new test case here

## Input Data Validation


As the famous software saying goes: Garbage In, Garbage Out. 

Data validation tests ensure that we're not getting garbage in! 

Data validation tests looks very similar to unit tests, except they are testing your data rather than your functions!

In [ ]:
import pandas as pd
loans = pd.read_csv('data3_loans.csv')

Let's add a simple check that we have the correct columns in our dataset, in the correct order. 

**One of the most common bugs in any data processing script is missing columns. By checking that the correct columns exist, and are in the correct order up-front, we can avoid much harder to understand and debug issues later on.**

In [ ]:
assert loans.columns.tolist() == ['issue_date','income_category','annual_income','loan_amount','term','purpose','interest_payments','loan_condition','interest_rate','total_pymnt','total_rec_prncp'], 'Incorrect columns'


When we are checking that simple values are equal to eachother (e.g. `1 + 2 == 3`), we can just use the `==` operator. However, with `pandas` objects, we need to do a bit more work to make sure that values are equal. 

In [ ]:
# This should fail
df1 = pd.DataFrame({'A': [1, 2, 3]})
df2 = pd.DataFrame({'A': [1, 2, 3]})
assert df1 == df2, 'This should fail. Dataframes cannot be compared with =='

In [ ]:
# This should pass
df1 = pd.DataFrame({'A': [1, 2, 3]})
df2 = pd.DataFrame({'A': [1, 2, 3]})
assert df1.equals(df2), "The dataframes are not equal"

We can also make assertions about the actual _contents_ of our data. Let's imagine that the loans data we're processing is from before the year 2011. We can create a check for this easily.

In [ ]:
assert (pd.to_datetime(loans['issue_date'], format='mixed') < pd.to_datetime('1-1-2012')).all(), 'There is a date from after 2011.'

As you can see, there is a date from after 2011! Again, catching this now, before processing our data, will result in less weird error messages, or in some cases stop a totally incorrect process from running! 

🧑‍💻 Let's write a few tests that validate the structure of our data:
- Check that the `income_category` is Low, Medium or High
- Check that there are no nan values in `loan_amount`
- Think of another reasonable test to make of this data

In [ ]:
# TODO: Write your new tests here

## Material vs. immaterial

Sadly, floating point numbers in Python don't always play nice with our asserts. Check out the following assert that seems like it should be true!

In [ ]:
assert .1 + .1 + .1 == .3

This occurs because floating point numbers are not exactly represented internally within computers. This is not an issue for most use cases, but it does mean we have to take special care when writing tests to assert that there are no _material_ differences in our expected and actual values.

Use the `math.isclose` function to check floating point numbers are close. 

In [ ]:
import math
assert math.isclose(.1 + .1 + .1, .3, abs_tol=1e-6), 'This should not print. Numbers are equal.'

If we want to compare numbers within some tolerance across entire dataframes, we can use the `assert_frame_equal` utility function given to us by the `pandas` library.

In [ ]:
# Create the first DataFrame
df1 = pd.DataFrame({'A': [1.0, 2.0, 3.0], 'B': [0.1, 0.2, 0.3]})

# Create the second DataFrame with a slight difference in floating-point values
df2 = pd.DataFrame({'A': [1.0, 2.0, 3.0], 'B': [0.100001, 0.2, 0.300001]})

In [ ]:
# Compare the dataframes directly. This should error.
assert df1.equals(df2), "The dataframes are not equal"

In [ ]:
# Compare the DataFrames while ignoring small differences in floating-point numbers. By default, 
# checks that numbers are within 1e-8 of eachother, but this can be configured by the atol={tolerance} 
# parameter

from pandas.testing import assert_frame_equal
assert_frame_equal(df1, df2, atol=1e-8)

## Integration Testing


The most expressive test you can build is an integration test. An integration test ensures that the fully-integrated process, from start to finish, works as you expect it to. 

Let's imagine we've used Python to automate the calculation of compounding interest with a `calculate_compound_interest` function. This is our full process, so we can write some integration tests to check it performs correctly.

In [51]:
def calculate_compound_interest(principal, interest_rate, time):
    """
    Calculate compound interest given the principal, interest rate, and time.

    Args:
        principal (float): The initial amount of money.
        interest_rate (float): The interest rate per period.
        time (int): The number of periods.

    Returns:
        float: The total amount after compounding the interest.
    """
    amount = principal * (1 + interest_rate) ** time
    return amount

In [52]:
import math

# Test with principal = $1000, interest_rate = 0.05, time = 2
assert math.isclose(calculate_compound_interest(1000, 0.05, 2), 1102.5)

# Test with principal = $5000, interest_rate = 0.08, time = 5
assert math.isclose(calculate_compound_interest(5000, 0.08, 5), 7346.640384000003)

🧑‍💻 Let's write a few more integration tests. As with unit tests, try to test both common cases, and edge cases. Some ideas:
1. What if the number of periods is 0?
2. What if the interest rate is 0, or negative?

In all of these cases, creating _expected_ results might be the challenging part -- maybe use Excel?

In [ ]:
# TODO: add your code here

## Creating Expected Results

As you've likely noticed from the above integration tests, you need to have some _expected_ values to test against to write effective tests. For processes that are way more complex than calculating compound interest, you likely cannot turn to another tool to generate expected values easily. So what do?

Since we're automating processes using Python, the likely location for expected values is the previous, manual version of the process you are automating. If you have an Excel file that implements the process you are automating and outputs an `output` tab -- create a test that tries to replicate the output tab!

If there is no manual version of the process and you are automating it from scratch, it's a good idea to build from unit tests to integration tests, checking your work manually along the way before building a final expected output you can be confident in.